In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# importing all neccesary libraries

In [2]:
import os
import csv
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow

from tensorflow import keras 
from keras import models
from keras import applications
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import *
from keras.models import *
from keras import backend as K
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [3]:
train  = pd.read_csv('../input/digit-recognizer/train.csv')
test   = pd.read_csv('../input/digit-recognizer/test.csv')
sample = pd.read_csv('../input/digit-recognizer/sample_submission.csv')

X = np.array(train.iloc[:, 1:])
Y = np.array(train.iloc[:, 0])
test = test.to_numpy()

n = X.shape[0]
m = X.shape[1]

print("Number of training samples: ", n)
print ("Number of features: ", m)

In [4]:
index = np.random.randint(0,n, 50)
plt.figure(figsize = (20, 10))
for i, idx in enumerate(index):
    plt.subplot(5, 10, i+1)
    plt.imshow(np.resize(X[idx], (28, 28, 1)))
    plt.axis("off")
print("Random examples".center(150))   
plt.show()

In [5]:
x_labels = sorted(list(set(Y)))
y_labels = [Y.tolist().count(i) for i in x_labels]

plt.figure(figsize = (12,5))
plt.ylabel('Number of labels',  fontsize=20)
plt.xlabel('Labels',  fontsize=20)
plt.xticks([i for i in range(len(x_labels))]) 
plt.bar(x_labels, y_labels)
plt.show()

# Data pre-proccesing

In [6]:
X = np.reshape(X, (X.shape[0], 28, 28, 1))
test = np.reshape(test, (test.shape[0], 28, 28, 1))

print('Dimensions of training data after resizing data: ', X.shape)
print('Dimensions of test data after resizing data: ', test.shape)

In [7]:
Y = to_categorical(Y)

print('Example output after one-hot encoding: ', Y[0])

In [8]:
X = X / 255.0
test = test / 255.0

print('Minimum value of the features before normalization: ', np.min(X))
print('Maximum value of features before normalization: ', np.max(X))

In [9]:
indexs = np.arange(n)
posicion = int(n * 0.8)

X_train_p = X[indexs[:posicion]]
X_train_v = X[indexs[posicion:]] 
Y_train_p = Y[indexs[:posicion]]
Y_train_v = Y[indexs[posicion:]]

print(X_train_v.shape)
print("Number of examples of X_train_p: ", X_train_p.shape[0])
print("Number of examples of X_train_v: ", X_train_v.shape[0])

In [10]:
data_gen = ImageDataGenerator(validation_split = 0.2, rotation_range = 10, zoom_range = 0.1, width_shift_range = 0.1, height_shift_range = 0.1)

In [11]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

model = keras.Sequential([
    # Block One
    layers.InputLayer(input_shape=[28, 28, 1]),
    
    # Data Augmentation
    # ____,
    #preprocessing.RandomContrast(factor=0.10),
    #preprocessing.RandomFlip(mode='horizontal'),
    #preprocessing.RandomRotation(factor=0.10),

    # Block One
    layers.Conv2D(32, kernel_size  = (5, 5), padding = 'same', activation = 'relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, kernel_size=(5,5), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size  = (2, 2)),
    layers.Dropout(0.5),

    # Block Two
    layers.Conv2D(filters=128, kernel_size=(5,5), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(filters=128, kernel_size=(5,5), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size  = (2, 2)),
    layers.Dropout(0.5),
    
    # Block Three
    layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same'),
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dropout(0.5),
    # Head
    layers.Dense(512, activation = 'relu'),
    layers.BatchNormalization(renorm=True),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax'),
])


In [12]:
optimizer = tf.keras.optimizers.Adam(epsilon=0.01)
model.compile(
    optimizer= optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

from tensorflow.keras import callbacks
# YOUR CODE HERE: define an early stopping callback
early_stopping = callbacks.EarlyStopping(
    #min_delta=0.001,
    monitor='val_loss',
    patience = 5,
    restore_best_weights=True,
    verbose=1
)

In [13]:
model.summary()

In [ ]:
history = model.fit(
    X_train_p, Y_train_p,
    validation_data=(X_train_v, Y_train_v),
    batch_size=500,
    epochs=20,
    callbacks=[early_stopping]
)
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

In [ ]:
pred = model.predict(test)
predictions = np.argmax(pred,axis=1)

In [ ]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)), "Label": predictions})
submissions.to_csv("submission.csv", index=False, header=True)